In [1]:
import socket

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import diffusion
from diffusion import SpacedDiffusion
from models import DiT_models
from download import find_model

from torch import Tensor
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
from utils import *

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [2]:
diff = diffusion.create_diffusion("")

In [3]:
model = DiT_models['DiT-S/2'](input_size=32, num_classes=1000).to(device)
model.train()

PATH = "Checkpoints/0750000.pt"
torch.serialization.add_safe_globals([PATH])

state_dict = find_model(PATH)
model.load_state_dict(state_dict)

/Users/egecimsir/Desktop/GenAI Practical/Critical-Multitask-Batch-Sizes-in-Diffusion-Models/download.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.

<All keys matched successfully>

In [39]:
x = torch.rand(3, 2)
model = nn.Linear(2, 10, bias=False)
opt = optim.AdamW(model.parameters())
out = model(x)
loss = out.mean()
loss.backward()

In [41]:
grads = get_gradient_vector(model)
grads.shape

torch.Size([20])

In [99]:
g = torch.tensor([1., 3., 6.])
G = torch.tensor([0., 3., 4.])

torch.norm(g - G)**2 / torch.norm(G)**2

tensor(0.2000)

In [11]:
np.random.choice(np.arange(5), size=6)

array([3, 4, 3, 2, 0, 1])

In [6]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [12]:
np.random.choice(100, size=5, replace=False)

array([27, 19, 31, 69, 71])

6